In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import time

In [4]:
import pandas as pd

In [5]:
import re

# Task 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:
KEYWORDS = ['python', 'парсинг']
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [6]:
def find_keyword(URL = 'https://habr.com/ru/all/', KEYWORDS = ['python', 'парсинг', 'код', 'сша', 'программа', '2020']):
    url = requests.get(URL)
    soup = BeautifulSoup(url.text, 'html.parser')
    time.sleep(0.3)
    previews = soup.find_all('article', class_='post post_preview')
    df = pd.DataFrame()
    for preview in previews:
        article_text = preview.find_all('div', class_='post__body post__body_crop')
        for keyword in article_text:
            keyword_lower = keyword.text.lower()
            if any ([desired in keyword_lower for desired in KEYWORDS]):
                date = preview.find('span', class_='post__time').text
                title = re.sub("\n", '', preview.find('h2', class_='post__title').text)
                link = preview.find('a', class_='post__title_link').attrs.get('href')
                row = {'date': date, 'title': title, 'link': link}
                df = pd.concat([df, pd.DataFrame([row])])
    
    return df

find_keyword()

,date,title,link
0,вчера в 20:34,"Анализатор кода не прав, да здравствует анализ...",https://habr.com/ru/company/pvs-studio/blog/53...
0,вчера в 20:10,Делаем печатные ссылки кликабельными с помощью...,https://habr.com/ru/post/530850/
0,вчера в 19:39,"Сеньоров не хватает, или Статистика зарплат Ja...",https://habr.com/ru/company/headzio/blog/528560/
0,вчера в 17:40,Советы по продажам для технических стартапов,https://habr.com/ru/post/530456/
0,вчера в 17:26,Рисование собственных представлений (View) в A...,https://habr.com/ru/company/otus/blog/530618/
0,вчера в 16:38,Новое лицо для «Честного знака»: работы победи...,https://habr.com/ru/company/mailru/blog/530802/


# Task 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [9]:
def check_emails(emails = ['xxx@x.ru', 'yyy@y.com']):
    url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    rec = requests.post(url, json={'emailAddresses': emails}, headers={'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'})
    
    breaches_df = pd.DataFrame(rec.json()['breaches']).T.reset_index()
    breaches_df = breaches_df [['breachId', 'site', 'description', 'publishDate']]
    
    emails_ = pd.DataFrame(rec.json()['summary']).T.reset_index()
    emails_.rename(columns = {'index':'emails','breaches':'breachId'}, inplace = True)
    emails_df = emails_['breachId'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('breachId').join(emails_[['emails']], how='left')
    emails_df = emails_df[['emails', 'breachId']]
    
    final_df = emails_df.merge(breaches_df, on='breachId', how='left')
    final_df = final_df.drop(['breachId'], axis=1)
    cols = final_df.columns.tolist()
    final_df = final_df[['emails', 'site', 'publishDate', 'description']]

    return final_df


check_emails()

,emails,site,publishDate,description
0,xxx@x.ru,verifications.io,2019-03-28T00:00:00Z,Big data e-mail verification platform verifica...
1,xxx@x.ru,vk.com,2020-05-21T00:00:00Z,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,parapa.mail.ru,2017-02-14T00:00:00Z,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,vk.com,2016-10-29T00:00:00Z,Popular Russian social networking platform VKo...
4,xxx@x.ru,adobe.com,2016-10-21T00:00:00Z,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,cfire.mail.ru,2017-02-14T00:00:00Z,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,cdprojektred.com,2017-01-31T00:00:00Z,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,imesh.com,2016-10-23T00:00:00Z,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,verifications.io,2019-03-28T00:00:00Z,Big data e-mail verification platform verifica...
9,yyy@y.com,www.dangdang.com,2019-02-21T00:00:00Z,"This is a list of email addresses only, and as..."
